In [1]:
import os
import pandas as pd

import plotly.express as px

In [10]:
df1 = pd.read_csv("process_adaptation_results_add_rules.csv")
df2 = pd.read_csv("process_adaptation_results_generate_from_bpmn_rules.csv")
df = pd.concat([df1, df2], ignore_index=True)
df = df.reset_index(drop=True)

df1 = pd.read_csv("process_adaptation_results_add_rules_exception_handling.csv")
df1["process_adaptation"] = "Typing Error Handling"
df2 = pd.read_csv("process_adaptation_results_generate_from_bpmn_rules_exception_handling.csv")
df2["process_adaptation"] = "Typing Error Handling"
df = pd.concat([df, df1, df2], ignore_index=True)
df = df.reset_index(drop=True)


df1 = pd.read_csv("process_adaptation_results_add_rules_exception_handling_db_error.csv")
df1["process_adaptation"] = "Database Error Handling"
df2 = pd.read_csv("process_adaptation_results_generate_from_bpmn_rules_exception_handling_db_error.csv")
df2["process_adaptation"] = "Database Error Handling"
df_db_error = pd.concat([df1, df2], ignore_index=True)
df_db_error = df_db_error.reset_index(drop=True)
df = pd.concat([df, df_db_error], ignore_index=True)
df = df.reset_index(drop=True)

df = df.drop(columns=["session_id", "seed", "correct_matches", "correct_columns", "file_extracted", "mails_sent", ])
df["tour"] = df["tour"].apply(lambda x: x.replace("J09", " ").title())
df["rule_adaptation_method"] = df["rule_adaptation_method"].apply(lambda x: "BPMN model" if x == "generate_bpmn" else "Human")
df["process_adaptation"] = df["process_adaptation"].apply(lambda x: "Base rule" if x == "base_rule" else
                                                          "Special cases: area 0" if x == "0_values" else
                                                          "Special cases: area 500" if x == "500_values" else
                                                          "Special cases: area 900" if x == "900_values" else
                                                          "Excluded cities" if x == "city_values" else
                                                          "Extension: mandatory reading " if x == "extension_estimates" else
                                                          "Extension: direct mail" if x == "extension_mail" else x)
df = df.rename(
    columns={
        "process_adaptation": "Process adaptation",
        "rule_adaptation_method": "Frame source",
        "tour": "Tour",
        "all_correct": "Successful process execution",
    }
)
display(df.value_counts("Successful process execution"))
display(df.groupby(["Frame source", "Process adaptation"])["Successful process execution"].mean())

df_plot = df.groupby(["Frame source", ], as_index=False)["Successful process execution"].sum()
df_plot["Successful process execution"] = df_plot["Successful process execution"].apply(lambda x: round(x / 190, 4) * 100)
print(df_plot.to_latex(index=False, float_format="%.2f"))

fig = px.bar(
    df_plot,
    x="Frame source",
    y="Successful process execution",
    color="Frame source",
    #title="Process adaptation success rates depending on the frame source",
    labels={"Successful process execution": "Successful process executions (%)"},
    text="Successful process execution",
)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    width=800,
    height=500,
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5
    )
)

Successful process execution
True     357
False     23
Name: count, dtype: int64

Frame source  Process adaptation           
BPMN model    Base rule                        1.00
              Database Error Handling          1.00
              Excluded cities                  1.00
              Extension: direct mail           0.80
              Extension: mandatory reading     0.85
              Special cases: area 0            1.00
              Special cases: area 500          0.95
              Special cases: area 900          0.95
              Typing Error Handling            0.92
Human         Base rule                        0.90
              Database Error Handling          1.00
              Excluded cities                  0.95
              Extension: direct mail           0.95
              Extension: mandatory reading     0.80
              Special cases: area 0            1.00
              Special cases: area 500          0.90
              Special cases: area 900          0.95
              Typing Error Handling            0.96
Name: Successful pro

\begin{tabular}{lr}
\toprule
Frame source & Successful process execution \\
\midrule
BPMN model & 94.21 \\
Human & 93.68 \\
\bottomrule
\end{tabular}

